# Create images DataFrame

In [ ]:
# CREATE IMAGES DATAFRAME FROM ORDERS DATAFRAME AND DATA FROM ETI - BY IMAGE LIST

images_df = pd.DataFrame()
images_df["Order ID"] = orders_df["Order ID"]
images_df["images_list"] = orders_df["Order ID"].apply(env.eti_api.get_image_list_by_orderid)

images_df = pd.concat([orders_df.set_index('Order ID'),
                       images_df.set_index('Order ID')], axis=1, join='inner').reset_index()

images_df = images_df.explode('images_list').rename(columns={'images_list': 'Image_ID'}).reset_index(drop=True)

# images_data = images_df["Image_ID"].apply(env.eti_api.get_image_list_by_orderid)


In [ ]:
# ADD IMAGES DATA BY ORDER ID FROM ETI (without splitting list of orders)

import tqdm.notebook as tq
import warnings
example_images_df = env.eti_api.get_images_data_by_orderid(orders_list[0])['images']
images_df = pd.DataFrame(columns=example_images_df[0].keys())
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)
    for order in tq.tqdm(orders_list, leave=False):
        order_df = pd.DataFrame(env.eti_api.get_images_data_by_orderid(order)['images'])
        images_df = pd.concat([images_df, order_df], axis='rows', ignore_index=True)

# Data exploration

In [ ]:
env.eti_api.get_categories_hierarchy()

In [ ]:
# SHOW RANDOM IMAGE FROM THE DATAFRAME

example_image_id = random.sample(list(top_weeds_df['imageID']), 1)[0]
image_data = top_weeds_df[top_weeds_df['imageID'] == example_image_id].reset_index()
image_num_tags = image_data.at[0, 'num_weed_tags']
image_crop_name = image_data.at[0, 'cropName']

im_path = env.download_image(example_image_id)
image = io.imread(im_path)

plt.imshow(image)
plt.title(f"Image ID: {example_image_id}\nCrop Type: {image_crop_name}, Weed Tags: {image_num_tags}")
plt.show()

In [ ]:
wide_im_path = env.download_image(matching_wide_image_id)
wide_image = io.imread(wide_im_path)

plt.imshow(wide_image)
plt.show()

In [ ]:
# ADD WIDE IMAGE_ID TO EACH ZOOM IMAGE
example_wide_image = env.eti_api.get_matching_wide_images([7275584])
# wide_images = env.eti_api.get_matching_wide_images(list(top_weeds_df['imageID']))
wide_im_path = env.download_image(example_wide_image[0])
wide_image = io.imread(wide_im_path)

plt.imshow(wide_image)
plt.show()

In [ ]:
# ADD WIDE IMAGE_ID TO EACH ZOOM IMAGE
example_wide_image = env.eti_api.get_matching_wide_images([7275584])
# wide_images = env.eti_api.get_matching_wide_images(list(top_weeds_df['imageID']))
wide_im_path = env.download_image(example_wide_image[0])
wide_image = io.imread(wide_im_path)

plt.imshow(wide_image)
plt.show()

In [ ]:
# VIEW AND FIND INFORMATION ABOUT NUMBER OF WEED TAGS IN THE IMAGES
images_with_weeds_df = images_df[images_df['num_weed_tags']]
print(len(images_with_weeds_df))
plt.hist(images_with_weeds_df['num_weed_tags'], bins=100)
plt.show()

In [ ]:
plt.hist(filtered_images_df['cameraAngle'], bins=100)
plt.show()

In [ ]:
plt.hist(images_df['cameraAngle'], bins=100)
plt.show()

In [ ]:
# images_with_the_most_weeds_tags['num_weed_tags']
plt.hist(top_weeds_soy['num_weed_tags'], bins=100)
plt.show()

In [ ]:
images_df.columns

In [ ]:
example_orderid = random.sample(orders_list, 1)[0]

im_list = env.eti_api.get_image_list_by_orderid(example_orderid, [2])
example_image_id = random.sample(im_list, 1)[0]

im_path = env.download_image(example_image_id)
image = io.imread(im_path)

plt.imshow(image)
plt.show()

In [ ]:
categories_hierarchy = env.eti_api.get_categories_hierarchy()


In [ ]:
for i in range(len(categories_hierarchy)):
    print(f"{i}, category: {categories_hierarchy[i]['id']}")

In [ ]:
# ADD CATEGORIES NAME AND TYPES TO IMAGES DATAFRAME

def create_cat_dict(cat_list):
    cat_dict_types = {}
    cat_dict_names = {}
    for l in cat_list:
        if 'type' in l:
            cat_dict_types[l['id']] = l['type']
        else:
            cat_dict_types[l['id']] = 'NoType'
        if 'name' in l:
            cat_dict_names[l['id']] = l['name']
        else:
            cat_dict_names[l['id']] = 'NoName'
    return cat_dict_types, cat_dict_names



def get_weed_subcategory_name_by_subcategory_id(types_ids_list):
    types_ids_list = eval(types_ids_list)
    print(types_ids_list)
    cat_dict_types, cat_dict_names = create_cat_dict(categories_hierarchy[9]['subCategories'])
    names_list = []
    types_list = []
    print(len(types_ids_list))
    for i in range(len(types_ids_list)):
        print(cat_dict_names[types_ids_list[i]])
        names_list.append(cat_dict_names[types_ids_list[i]])
        types_list.append(cat_dict_types[types_ids_list[i]])

    
    
    # weed_subcategories = categories_hierarchy[9]['subCategories']
    # names_list = []
    # for i in range(len(types_ids_list)):
    #     for j in range(len(weed_subcategories)):
    #         if weed_subcategories[j]['id'] == types_ids_list[i]:
    #             names_list.append(weed_subcategories[j]['name'])

    # if len(names_list) != len(types_ids_list):
        # raise Exception("names_list and types_ids_list lengths are different")
    return names_list

images_df[images_df['weed_types_ids_list'].apply(get_weed_subcategory_name_by_subcategory_id)]
# print(images_df[images_df['weed_types_ids_list'].apply(get_weed_subcategory_name_by_subcategory_id)])
# images_df['weed_types_list'] = images_df[images_df['weed_types_ids_list'].apply(get_weed_subcategory_name_by_subcategory_id)]

In [ ]:
images_df = pd.read_csv("/mnt/disks/datasets/wide_images/images_df_new_1.csv")
unique_weed_types = np.unique(images_df['weed_types_dict'].explode(), return_counts=True)